In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from gensim.models import KeyedVectors
model_w2v = KeyedVectors.load_word2vec_format('drive/MyDrive/CS626/Assignment_2/GoogleNews-vectors-negative300.bin',binary=True)

In [ ]:
from nltk.corpus import semcor
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('semcor')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import re
import warnings
from sklearn import metrics
from nltk import word_tokenize
from numpy import dot
from numpy.linalg import norm
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
semcor_tagged_sents = semcor.tagged_sents(tag='sem')

In [ ]:
semcor_tagged_sents[0]

[['The'],
 Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]),
 Tree(Lemma('state.v.01.say'), ['said']),
 Tree(Lemma('friday.n.01.Friday'), ['Friday']),
 ['an'],
 Tree(Lemma('probe.n.01.investigation'), ['investigation']),
 ['of'],
 Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']),
 ["'s"],
 Tree(Lemma('late.s.03.recent'), ['recent']),
 Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']),
 Tree(Lemma('produce.v.04.produce'), ['produced']),
 ['``'],
 ['no'],
 Tree(Lemma('evidence.n.01.evidence'), ['evidence']),
 ["''"],
 ['that'],
 ['any'],
 Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']),
 Tree(Lemma('happen.v.01.take_place'), ['took', 'place']),
 ['.']]

In [ ]:
vectors = {}
semcor_sents = semcor.sents()
for sentence in semcor_sents:
    for word in sentence:
        if word in model_w2v:
            vectors[word] = model_w2v.get_vector(word)

In [ ]:
def sent_vec(sentence):
    li = []
    for word in sentence:
        if word in vectors:
            li.append(vectors[word])
    if len(li) == 0:
        return np.zeros(300)
    return np.average(li,axis=0)

In [ ]:
def get_sense_dict(sents):
    sense_set = set()
    for i in range(len(sents)):
        for j in range(len(sents[i])):
            if isinstance(sents[i][j], nltk.Tree):
                try :
                    if sents[i][j].height() == 3:
                        for tree in sents[i][j]:
                            if(tree.label() == 'NE'):
                                sense = 'NE'
                                sense_set.add(sense) 
                    else :
                        sense = sents[i][j].label().synset().name()
                        sense_set.add(sense)
 
                except:
                    if(sents[i][j].label() == 'NE'):
                        sense = sents[i][j].label()
                        sense_set.add(sense)
                    else :
                        sense = sents[i][j].label()
                        sense_set.add(sense)
            else :
                sense = 'notag'
                sense_set.add(sense)
    sense_set.add('unk')
    sense_set = list(sense_set)
    sense_dict = {sense : i for i, sense in enumerate(sense_set)}
    return sense_dict

In [ ]:
stop_words = set(stopwords.words('english'))

def predict_synset_extended(word, sentence):
    sentence = [w for w in sentence if (w not in stop_words) and (w.isalnum())]

    if word in sentence:
        sentence.remove(word)
        
    context_bag = sent_vec(sentence)
    
    
    sense_bag = {}
    senses = wn.synsets(word)

    if len(senses)>0:
        for sense in senses:
            sense_bag[sense] = []
            sense_bag[sense].append(sent_vec([w for w in sense.definition().split() if (w not in stop_words) and (w.isalnum())]))

            for hypo in sense.hyponyms():
                sense_bag[sense].append(sent_vec([w for w in hypo.definition().split() if (w not in stop_words) and (w.isalnum())]))
        
        if len(sense_bag.keys()) > 0:
            synset = ""

            cos_sims = []
            for key,val in sense_bag.items():
                cos_sims.append((key,model_w2v.cosine_similarities(context_bag,val).mean()))#,model_w2v.cosine_similarities(context_bag,val)))
            cos_sims.sort(key = lambda x: x[1],reverse=True)
            return str(cos_sims[0][0])[8:-2]
    else:
        return "notag"


In [ ]:
def predict(sents, untagged_sents):
    pred_sense = []
    actual_sense = []
    for i in range(len(sents)):
        for j in range(len(sents[i])):
            if isinstance(sents[i][j], nltk.Tree):
                try :
                    if sents[i][j].height() == 3:
                        for tree in sents[i][j]:
                            if(tree.label() == 'NE'):
                                sense = 'NE'
                                actual_sense.append(sense)
                                word = "_".join(tree.leaves())
                    else :
                        sense = sents[i][j].label().synset().name()
                        actual_sense.append(sense)
                        word = "_".join(sents[i][j].leaves())
                    pred_sense.append(predict_synset_extended(word, untagged_sents[i]))                   
                except:
                    if(sents[i][j].label() == 'NE'):
                        sense = sents[i][j].label()
                        word = "_".join(sents[i][j].leaves())
                        actual_sense.append(sense)
                    else :
                        sense = sents[i][j].label()
                        word = "_".join(sents[i][j].leaves())
                        actual_sense.append(sense)
                    pred_sense.append(predict_synset_extended(word, untagged_sents[i]))                
            else :
                sense = 'notag'
                actual_sense.append(sense)
                word = "_".join(sents[i][j])
                pred_sense.append(predict_synset_extended(word, untagged_sents[i]))           
    return actual_sense, pred_sense

In [ ]:
sents = semcor.tagged_sents(tag='sem')
sense_dict = get_sense_dict(sents)
untagged_sents = semcor.sents()
y_true, y_pred = predict(sents, untagged_sents)

y_true = [sense_dict[sense] for sense in y_true]
y_pred = [sense_dict.get(sense, sense_dict['notag']) for sense in y_pred]


In [ ]:
inv_map = {v: k for k, v in sense_dict.items()}
y_true = [inv_map[i] for i in y_true]
y_pred = [inv_map[i] for i in y_pred]
y_true_n, y_pred_n = [],[]
for i in range(len(y_true)):
  if(y_true[i] != 'notag' and y_true[i] != 'NE'):
    temp = y_true[i].split('.')[1]
    if(temp=='n'):
      y_true_n.append(y_true[i])
      y_pred_n.append(y_pred[i])

In [ ]:
print("Accuracy : ", metrics.accuracy_score(y_true_n, y_pred_n))
precision, recall, f1score, _ = metrics.precision_recall_fscore_support(y_true, y_pred, average='weighted',zero_division=0)
print("Precision : ", precision)
print("Recall : ", recall)
print("F1 Score : ", f1score)

Accuracy :  0.4207885214277609
Precision :  0.7172896749077547
Recall :  0.5928444733857615
F1 Score :  0.6267663941848407


In [ ]:
test_sentence = "She likes to play cricket"
tokens = word_tokenize(test_sentence)
for word in tokens:
  pred_sense = predict_synset_extended(word, tokens)
  if(pred_sense!='notag'):
    print("Target Word : " + word + "\nPredicted Sense : " + pred_sense + " : " + wn.synset(pred_sense).definition() + "\n")
  else:
    print("Target Word : " + word + "\nPredicted Sense : " + pred_sense + "\n")

Target Word : She
Predicted Sense : notag

Target Word : likes
Predicted Sense : like.n.02 : a kind of person

Target Word : to
Predicted Sense : notag

Target Word : play
Predicted Sense : playing_period.n.01 : (in games or plays or other performances) the time during which play proceeds

Target Word : cricket
Predicted Sense : cricket.n.02 : a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs

